In [11]:
import re
import pandas as pd
import requests 
import json

### Starting of classification Tag evaluation API

In [12]:
# raw_corpus = pd.read_csv('/home/ifte/resource/docomo_dic/data/corpus.csv')
# raw_corpus = pd.read_csv('/home/iftekhar/docomodezwhu/5495/data/corpus.csv')
raw_corpus = pd.read_csv('/home/hduser_/Downloads/1810/data/corpus.csv')

### Converter DocID => URLs

In [4]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
data = {
# "output_dir": "/home/ifte/resource/docomo_dic/model", 
# "output_dir": "/home/iftekhar/docomodezwhu/model", 
"output_dir": "/home/hduser_/Downloads/1810/model/",    
"agent_id": -1, 
"query": "メールサービスの利用にあたっての注意点を教えてください。",
"classification": "dトラベル",
"max_answer": 5, 
# "data_dir": "/home/ifte/resource/docomo_dic/data", 
"data_dir": "/home/hduser_/Downloads/1810/data", 
"keyphrase_extraction": 0
}

r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 



In [5]:
# json.loads(r.text)
for i in json.loads(r.text)['page_relevance']:
    print(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0], i['doc_id'])


list(set(raw_corpus[raw_corpus['page'] == 760].url.values))[0]


https://travel.dmkt-sp.jp/faq/106/ 941
https://travel.dmkt-sp.jp/faq/176/ 924
https://travel.dmkt-sp.jp/faq/21/ 763
https://travel.dmkt-sp.jp/faq/82/ 1080
https://travel.dmkt-sp.jp/faq/82 1020


In [52]:
' '.join(raw_corpus[raw_corpus['page'] == 760].text.values)

'宿泊代金の支払方法について教えてください。 宿泊代金の支払方法について教えてください。|よくあるご質問 Q.宿泊代金の支払方法について教えてください。 A. 商品によって異なります。以下をご確認ください。 【宿提供マークのついているプラン】 宿泊当日に宿泊施設現地でお支払いください。 現金以外のお支払い方法の可否については宿泊施設へお問い合わせください。 ※各施設で利用できるクレジットカードについては各施設の「宿・ホテル詳細」をご覧下さい。 【手配マークのついているプラン】／【エースJTBマークのついているプラン】／【サン＆サンマークのついているプラン】 ドコモ払い、dカード払い、クレジットカード払いをご利用いただけます。 ※ｄトラベルでは海外発行のクレジットカードのご利用はできません。あらかじめご了承ください。 ※iD、d払いはご利用いただけません。 キーワード検索 お困りの内容 (c)NTT DOCOMO, INC. All Rights Reserved.'

In [13]:
def mean_reciprocal_rank_score(actual_value, predicted_values):
    if not actual_value:
        return 0
    pos = 0
    val = 0
    for i in predicted_values:
        if i == actual_value and pos == 0:
            val = 1
            break
        elif i == actual_value and pos == 1:
            val = 0.5
            break
        elif i == actual_value and pos == 2:
            val = 0.33
            break
        else:
            val = 0
        pos += 1

    return val

def cut_string(text):
    try:
        # print(text[:text.index("?")])
        return text[:text.index("?")]
    except (ValueError, AttributeError):
        # print(text)
        return text

In [14]:
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020_red.csv'
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020.csv'
# test_data = '/home/ifte/Documents/test_data_docomo_nai_form.csv'
# test_data = '/home/ifte/Documents/test_data_few_exceptional.csv'
# test_data = '/home/iftekhar/Documents/test_data_docomo_21.12.2020.csv'
# test_data = '/home/hduser_/Downloads/test_data_latest_Docomo_OOC240_successful.csv'
# test_data = '/home/hduser_/Downloads/test_data_latest_Docomo_OOC240_unsuccessful.csv'
test_data = '/home/hduser_/Downloads/test_data_latest_Docomo_OOC240_successful_headline_check.csv'
test_df = pd.read_csv(test_data)

In [15]:
test_df.rename(columns={'correct_url': 'url'}, inplace=True)

test_df.url = test_df.url.apply(lambda x: cut_string(x))

In [ ]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
i = 1
for index, col in test_df.iterrows():
    # data to be sent to api 
#     data = {'output_dir':'/home/ifte/resource/docomo_dic/model', 
#             'data_dir':'/home/ifte/resource/docomo_dic/data', 
#             'agent_id':-1, 
#             'query':col['question'],
#             'classification': col['tag'],
#             'max_answer':5
#            } 

#     data = {'output_dir':'/home/iftekhar/docomodezwhu/5495/model', 
#             'data_dir':'/home/iftekhar/docomodezwhu/5495/data', 
#             'agent_id':-1, 
#             'query':col['question'],
#             'classification': col['tag'],
#             'max_answer':5
#            } 
    
    data = {'output_dir':'/home/hduser_/Downloads/1810/model', 
        'data_dir':'/home/hduser_/Downloads/1810/data', 
        'agent_id':-1, 
        'query':col['question'],
        'classification': col['tag'],
        'max_answer':5
       } 
    
    r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers)
    print('Request count:', i)
    i += 1
    result.append(r.text)
    

Request count: 1
Request count: 2


In [ ]:
result[46]

In [7]:
from collections import defaultdict
saver = defaultdict(list)

j = 0
for items in result:
    if 'Error: Failed' not in items or 'Error: Response' not in items:
        val = json.loads(items)['page_relevance']
        for i in val:
            saver[j].append(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0])
            saver[str(str(j)+'doc_id')].append(i['doc_id'])
#         print('page: ', j)
    j += 1
        

In [8]:
collector = defaultdict(list)
i = 0

for index, col in test_df.iterrows():
    collector['question'].append(col['question'])
    collector['tag'].append(col['tag'])
    collector['correct_url'].append(col['url'])
    collector_correct = []
    try:
        collector_correct = list(set(raw_corpus[raw_corpus['url'] == col['url']].page.values))[0]
    except IndexError:
        continue
        pass
    collector['correct_id'].append(collector_correct)
    collector['returned_id'].append(saver[str(str(i)+'doc_id')])
    score = mean_reciprocal_rank_score(collector_correct, saver[str(str(i)+'doc_id')])
    if score:
        collector['mrr_score'].append(score)
    else:
        try:
            url_loc = saver[i].index(col['url'])
            if url_loc == 1:
                collector['mrr_score'].append(0.5)
            elif url_loc == 2:
                collector['mrr_score'].append(0.33)
            elif url_loc == 0:
                collector['mrr_score'].append(1)
            elif url_loc >= 3:
                collector['mrr_score'].append('LOW_RANK')
    
        except ValueError:
            collector['mrr_score'].append('NOT_MATCHED')
            pass

    collector['urls'].append(saver[i])
#         print('ok ', i)
    i += 1

    

In [9]:
new_df = pd.DataFrame.from_dict(collector, orient='index').T

# new_df.to_csv('nai_form_with_classification_tag_23.12.2020.csv')

new_df.to_csv('/home/iftekhar/Downloads/test_data_latest_Docomo_OOC240_result_BMreplace_fasttext_model_300.csv')